Analysing the Medicaid and Chip Eligibilty

Import pandas Libriries

In [1]:
import pandas as pd

In [2]:
#convert the Data Dictionary csv file into .md file

In [3]:
Data_Dictionary = pd.read_csv("../data/Data_Dictionary.csv")
Data_Dictionary = Data_Dictionary.drop(columns=['name','type','format'])
#Convert the data frame into a mark down string(Github style)
Data_Dictionary = Data_Dictionary.to_markdown(tablefmt="github", index=False)
# Export the Markdown string to a .md file
output_path = "../Data_Dictionary.md"
with open(output_path, 'w', encoding='utf-8') as f:
  f.write(Data_Dictionary)

In [4]:
# Read the csv data into medicaid dataframe
medicaid = pd.read_csv("../data/medicaid.csv")
medicaid.head()

,State Abbreviation,State Name,Reporting Period,State Expanded Medicaid,Preliminary or Updated,Final Report,New Applications Submitted to Medicaid and CHIP Agencies,New Applications Submitted to Medicaid and CHIP Agencies - footnotes,Applications for Financial Assistance Submitted to the State Based Marketplace,Applications for Financial Assistance Submitted to the State Based Marketplace - footnotes,...,Total Medicaid and CHIP Determinations Processed between 31 days and 45 days,Total Medicaid and CHIP Determinations Processed between 31 days and 45 days - footnotes,Total Medicaid and CHIP Determinations Processed in More than 45 Days,Total Medicaid and CHIP Determinations Processed in More than 45 Days - footnotes,Total Call Center Volume (Number of Calls),Total Call Center Volume (Number of Calls) - footnotes,Average Call Center Wait Time (Minutes),Average Call Center Wait Time (Minutes) - footnotes,Average Call Center Abandonment Rate,Average Call Center Abandonment Rate - footnotes
0,AK,Alaska,201309,N,U,Y,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AK,Alaska,201706,Y,P,N,3017.0,Includes Renewals and/or Redeterminations,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AK,Alaska,201706,Y,U,Y,3069.0,Includes Renewals and/or Redeterminations,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AK,Alaska,201707,Y,P,N,3187.0,Includes Renewals and/or Redeterminations,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AK,Alaska,201707,Y,U,Y,3265.0,Includes Renewals and/or Redeterminations,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Clean the Data

In [5]:
#Clean column names
medicaid.columns = (
    medicaid.columns
    .str.lower()  #lower the alll column names
    .str.strip()  #Remove extra spaces
    .str.replace(' ','_') #replace space with underscore
    .str.replace(r"_+", '_') #replace multiple underscores, if have any
    .str.replace(r"_-_footnotes", '_footnotes')
)

In [6]:
#Change the data type of reporting_period and extract month
medicaid['reporting_period'] = pd.to_datetime(medicaid['reporting_period'].astype(str),
    format='%Y%m'
    )
medicaid['month_name']= medicaid['reporting_period'].dt.month_name()
medicaid['year'] = medicaid['reporting_period'].dt.year

In [7]:
#reorder columns
medicaid.insert(3, 'year', medicaid.pop('year'))  # pop removes the coumn from the location and insert allows to insert column in specific location 
medicaid.insert(4, 'month_name', medicaid.pop('month_name'))

In [14]:
#Remove duplicate rows for state and reporting period
medicaid_clean = (
    medicaid
    .sort_values("preliminary_or_updated") #Sort values for preliminary_or_updated
    .drop_duplicates(subset=['state_abbreviation', 'reporting_period'], keep='last') #Make sure to keep the U (Upadated records) if there are priliminary and updated
    .sort_values(["state_abbreviation","reporting_period"])
)
medicaid_clean

,state_abbreviation,state_name,reporting_period,year,month_name,state_expanded_medicaid,preliminary_or_updated,final_report,new_applications_submitted_to_medicaid_and_chip_agencies,new_applications_submitted_to_medicaid_and_chip_agencies_footnotes,...,total_medicaid_and_chip_determinations_processed_between_31_days_and_45_days,total_medicaid_and_chip_determinations_processed_between_31_days_and_45_days_footnotes,total_medicaid_and_chip_determinations_processed_in_more_than_45_days,total_medicaid_and_chip_determinations_processed_in_more_than_45_days_footnotes,total_call_center_volume_(number_of_calls),total_call_center_volume_(number_of_calls)_footnotes,average_call_center_wait_time_(minutes),average_call_center_wait_time_(minutes)_footnotes,average_call_center_abandonment_rate,average_call_center_abandonment_rate_footnotes
0,AK,Alaska,2013-09-01,2013,September,N,U,Y,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AK,Alaska,2017-06-01,2017,June,Y,U,Y,3069.0,Includes Renewals and/or Redeterminations,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AK,Alaska,2017-07-01,2017,July,Y,U,Y,3265.0,Includes Renewals and/or Redeterminations,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AK,Alaska,2017-08-01,2017,August,Y,U,Y,3967.0,Includes Renewals and/or Redeterminations,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AK,Alaska,2017-09-01,2017,September,Y,U,Y,3553.0,Includes Renewals and/or Redeterminations,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10294,WY,Wyoming,2025-06-01,2025,June,N,U,Y,1429.0,Includes Administrative Data Transfers; Does N...,...,71.0,NaN,159.0,NaN,9879.0,NaN,1.0,Call centers offer callbacks,0.011,Callbacks are included
10296,WY,Wyoming,2025-07-01,2025,July,N,U,Y,1539.0,Includes Administrative Data Transfers; Does N...,...,115.0,NaN,170.0,NaN,11585.0,NaN,1.0,Call centers offer callbacks,0.002,Callbacks are included
10298,WY,Wyoming,2025-08-01,2025,August,N,U,Y,1396.0,Includes Administrative Data Transfers; Does N...,...,88.0,NaN,306.0,NaN,11046.0,NaN,1.0,Call centers offer callbacks,0.003,Callbacks are included
10300,WY,Wyoming,2025-09-01,2025,September,N,U,Y,1541.0,Includes Administrative Data Transfers; Does N...,...,114.0,NaN,175.0,NaN,12548.0,NaN,1.0,Call centers offer callbacks,0.004,Callbacks are included


In [38]:
#Drop rows for the year 2013 because it does not have any info and after 2013 sep there were no entries until 2017
medicaid_clean = medicaid_clean.drop(medicaid_clean[medicaid_clean["year"]== 2013].index)
medicaid_clean.shape

(5151, 46)

In [39]:
medicaid_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5151 entries, 2 to 10301
Data columns (total 46 columns):
 #   Column                                                                                    Non-Null Count  Dtype         
---  ------                                                                                    --------------  -----         
 0   state_abbreviation                                                                        5151 non-null   object        
 1   state_name                                                                                5151 non-null   object        
 2   reporting_period                                                                          5151 non-null   datetime64[ns]
 3   year                                                                                      5151 non-null   int32         
 4   month_name                                                                                5151 non-null   object        
 5   state_expa

In [42]:
#Drop all the footnotes columns
medicaid_drop_footnotes = medicaid_clean.drop(columns= medicaid_clean.columns[medicaid_clean.columns.str.contains("_footnotes")])

In [43]:
#export csv cleaned csv files
medicaid_drop_footnotes.to_csv("../data/medicaid_drop_footnotes.csv", index=False)
medicaid_clean.to_csv("../data/medicaid_chip.csv", index=False)

In [59]:
medicaid_clean[medicaid_clean["state_abbreviation"]== 'NY']["applications_for_financial_assistance_submitted_to_the_state_based_marketplace"].sum()

np.float64(32511134.0)